In [1]:
# %load_ext autoreload
# %autoreload 2
# import sys; sys.path.append('/future/u/okhattab/repos/public/stanfordnlp/dspy')

import dspy
from dspy.evaluate import Evaluate
from dspy.datasets.gsm8k import GSM8K, gsm8k_metric
from dspy.teleprompt import BootstrapFewShotWithRandomSearch
from dspy.modeling import JSONBackend, LiteLM

In [2]:
gms8k = GSM8K()

lm = LiteLM(model="gpt-3.5-turbo", default_params={"max_tokens": 500, "temperature": 0.5, "num_retries": 5})
backend = JSONBackend(lm=lm, attempts=3)
dspy.settings.configure(backend=backend)

trainset, devset = gms8k.train, gms8k.dev

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1319/1319 [00:00<00:00, 40417.06it/s]


In [3]:
NUM_THREADS = 4
evaluate = Evaluate(devset=devset[:], metric=gsm8k_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [4]:
class CoT(dspy.Module):
    def __init__(self):
        super().__init__()
        self.prog = dspy.ChainOfThought("question -> answer")
    
    def forward(self, question):
        return self.prog(question=question)

In [5]:
RUN_FROM_SCRATCH = True

if RUN_FROM_SCRATCH:
    config = dict(max_bootstrapped_demos=5, max_labeled_demos=8, num_candidate_programs=5, num_threads=NUM_THREADS)
    teleprompter = BootstrapFewShotWithRandomSearch(metric=gsm8k_metric, **config)
    cot_bs = teleprompter.compile(CoT(), trainset=trainset, valset=devset)
    # cot_bs.save('turbo_8_8_10_gsm8k_200_300.json')
else:
    cot_bs = CoT()
    cot_bs.load('turbo_8_8_10_gsm8k_200_300.json')

Going to sample between 1 and 5 traces per predictor.
Will attempt to train 5 candidate sets.


Average Metric: 174 / 300  (58.0): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 474.27it/s]


Average Metric: 174 / 300  (58.0%)
Score: 58.0 for set: [0]
New best score: 58.0 for seed -3
Scores so far: [58.0]
Best score: 58.0


Average Metric: 8.0 / 53  (15.1):  17%|████████████████████▎                                                                                                | 52/300 [00:00<00:00, 269.06it/s]

Error for example in dev set: 		 Generation failed, recursively attempts to complete did not succeed.


...

Average Metric: 144 / 300  (48.0%)
Score: 48.0 for set: [8]
Scores so far: [58.0, 17.67, 64.33, 67.67, 75.0, 71.67, 48.0]
Best score: 75.0
Average of max per entry across top 1 scores: 0.75
Average of max per entry across top 2 scores: 0.9166666666666666
Average of max per entry across top 3 scores: 0.9733333333333334
Average of max per entry across top 5 scores: 0.9866666666666667
Average of max per entry across top 8 scores: 0.9866666666666667
Average of max per entry across top 9999 scores: 0.9866666666666667


  1%|█▌                                                                                                                                                      | 2/200 [00:00<00:00, 557.57it/s]


Bootstrapped 2 full traces after 3 examples in round 0.


Average Metric: 137 / 300  (45.7): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 445.77it/s]


Average Metric: 137 / 300  (45.7%)
Score: 45.67 for set: [8]
Scores so far: [58.0, 17.67, 64.33, 67.67, 75.0, 71.67, 48.0, 45.67]
Best score: 75.0
Average of max per entry across top 1 scores: 0.75
Average of max per entry across top 2 scores: 0.9166666666666666
Average of max per entry across top 3 scores: 0.9733333333333334
Average of max per entry across top 5 scores: 0.9866666666666667
Average of max per entry across top 8 scores: 0.9933333333333333
Average of max per entry across top 9999 scores: 0.9933333333333333
8 candidate programs found.


In [6]:
evaluate(cot_bs, devset=devset[:])

Average Metric: 225 / 300  (75.0): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 300/300 [00:00<00:00, 432.23it/s]


Average Metric: 225 / 300  (75.0%)


75.0

In [7]:
print(backend.history[-1].prompt.to_str())

Given the fields `question`, produce the fields `answer`.

--

Provided the following:
Question: ${question}

Please return the following fields:
Reasoning: Let's think step by step in order to ${produce the answer}. We ...
Answer: ${answer}

According to the following JSON schema:
{"properties": {"question": {"title": "Question", "type": "string"}, "rationale": {"title": "Rationale", "type": "string"}, "answer": {"title": "Answer", "type": "string"}}, "required": ["question", "rationale", "answer"]}

--

{"question": "Nora is selling muffins to raise $120 for an upcoming school trip. She has packaged the muffins in cases. Each case has three packs of muffins. Each pack has four muffins. If she prices each muffin at two dollars, how many cases of muffins must she sell to raise enough money for the trip?", "rationale": "Let's think step by step in order to produce the answer. We know that each pack has 4 muffins and each case has 3 packs, so each case has 12 muffins. If each muffin is p